In [1]:
%reset -f
import os
import sys

PROJ_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.append(PROJ_PATH) 

import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import itertools
from pandas.api.types import CategoricalDtype

from datetime import date
from datetime import datetime, timedelta
import datetime as DT
from dateutil.relativedelta import relativedelta

import helper_functions as hfs
from scipy.stats import linregress
import math
import numba as nb

import boto3

warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)  # to see all columns of dataframe
pd.set_option("display.max_rows", None)  # to see all rows of dataframe
pd.set_option("display.max_colwidth", None)  # to see the full (un-truncated) dataframe field/cell value 

%precision %.3f 

'%.3f'

In [2]:
import configparser

env_config = configparser.ConfigParser()
env_config.read(f'{PROJ_PATH}/kye/env_config.ini')
# <<<<<<<<<<<<<<< >>>>>>>>>>>>>>>>>>>>>>> #
SPLIT_DATE = pd.to_datetime(env_config['ENV_VARS']['SPLIT_DATE'])
ML_DATA_FOLDER_PATH = env_config['LOCAL_PATHS']['ML_DATA_FOLDER_PATH']
PROCESSING_TEMP_FOLDER_PATH = env_config['LOCAL_PATHS']['PROCESSING_TEMP_FOLDER_PATH'] 
#@@@@@@ **********************<<<<<<<<<<<<#

print("SPLIT_DATE: ", SPLIT_DATE)

TODAY = str(date.today()) 
#FROM_DATE = pd.to_datetime(date.today(), format='%Y-%m-%d') - DT.timedelta(days=((365*4) + 100)) # "2016-01-01"
FROM_DATE = pd.to_datetime(date.today(), format='%Y-%m-%d') - DT.timedelta(days=((365*2) + 100)) # "2016-01-01"
TO_DATE = str(date.today())
TWO_YEARS_BEFORE_FROM_DATE = pd.to_datetime(FROM_DATE, format='%Y-%m-%d') - DT.timedelta(days=730)
FROM_DATE 

SPLIT_DATE:  2023-08-09 00:00:00


Timestamp('2021-05-01 00:00:00')

In [3]:
files_in_ML_DATA = os.listdir(f"{PROJ_PATH}/{ML_DATA_FOLDER_PATH}")
files_in_ML_DATA 

['all_stocks_ohlcv.pkl',
 'stocks_current_fundamentals_1340_2023-08-09.pkl',
 'indices_daily_ohlcv_2023-08-09.pkl',
 '.ipynb_checkpoints']

In [4]:
files_in_PROCESSING_TEMP = os.listdir(f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}") 
files_in_PROCESSING_TEMP

['RIPOs_df.pkl', 'indices_daily_ohlcv_2023-08-09.pkl', '.ipynb_checkpoints']

In [5]:
#for file in files_in_ML_DATA:
#    df = pd.read_csv( "s3://" + bucketname + "/" + ML_DATA_FOLDER_PATH + file) 
#    hfs.write_pickle_to_S3(df, bucketname, '/'+ ML_DATA_FOLDER_PATH + file.split(".")[0] + '.pkl') 

# Daily OHLCV

In [6]:
#stocks_daily_ohlcv = pd.read_pickle( "s3://" + bucketname + "/" + ML_DATA_FOLDER_PATH + [x for x in files_in_ML_DATA if 'stocks_daily_ohlcv_eligible' in x][0]) 
stocks_daily_ohlcv = pd.read_pickle(f"{PROJ_PATH}/{ML_DATA_FOLDER_PATH}/{[x for x in files_in_ML_DATA if 'all_stocks_ohlcv' in x][0]}") 

del stocks_daily_ohlcv['close_to_adj_close_ratio']
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv = stocks_daily_ohlcv[-stocks_daily_ohlcv.isna().any(axis=1)].reset_index(drop=True)
#del stocks_daily_ohlcv['close_to_adj_close_ratio']
#stocks_daily_ohlcv = stocks_daily_ohlcv[stocks_daily_ohlcv['Ticker'].isin([x for x in stocks_daily_ohlcv['Ticker'].unique()][0:10])].reset_index(drop=True)
print("Tickers: ", stocks_daily_ohlcv['Ticker'].nunique())
#stocks_daily_ohlcv.fillna(0,inplace=True)
now = datetime.now() 
stocks_daily_ohlcv = hfs.convert_floats_to_float32(stocks_daily_ohlcv)
print((datetime.now() - now).total_seconds()) 

print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.head(2) 

(755570, 9)
Tickers:  1340
0.056806
(755570, 9)


,company_Name,Ticker,Date,Adjusted_Open,Adjusted_High,Adjusted_Low,Adjusted_close,Adjusted_Volume,Exchange
0,Alcoa Corp,AA,2021-05-03,36.548000,37.408001,35.437000,36.971001,4738228.5,NYSE
1,Alcoa Corp,AA,2021-05-04,37.374001,38.603001,37.009998,38.405998,6820821.0,NYSE


In [7]:
stocks_daily_ohlcv["Date"] = hfs.str_to_date_col(stocks_daily_ohlcv[["Date"]]) 
stocks_daily_ohlcv = stocks_daily_ohlcv[stocks_daily_ohlcv["Date"] >= pd.to_datetime(FROM_DATE)].reset_index(drop=True)
# sorting by date
stocks_daily_ohlcv = stocks_daily_ohlcv.sort_values(by=['Ticker','Date']).reset_index(drop=True)
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.tail(2) 

(755570, 9)


,company_Name,Ticker,Date,Adjusted_Open,Adjusted_High,Adjusted_Low,Adjusted_close,Adjusted_Volume,Exchange
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ
755569,Zymeworks Inc. Common Stock,ZYME,2023-08-09,7.29,7.725,7.29,7.70,514001.0,NASDAQ


### Price Features:

In [8]:
stocks_daily_ohlcv = stocks_daily_ohlcv.rename(columns={"Adjusted_Open":"Adj_Open", "Adjusted_High": "Adj_High",
                                   "Adjusted_Low": "Adj_Low", "Adjusted_close": "Adj_Close" , 
                                    "Adjusted_Volume" : "Adj_Vol"})
stocks_daily_ohlcv.head(2) 

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange
0,Alcoa Corp,AA,2021-05-03,36.548000,37.408001,35.437000,36.971001,4738228.5,NYSE
1,Alcoa Corp,AA,2021-05-04,37.374001,38.603001,37.009998,38.405998,6820821.0,NYSE


In [9]:
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.head(2)

(755570, 9)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange
0,Alcoa Corp,AA,2021-05-03,36.548000,37.408001,35.437000,36.971001,4738228.5,NYSE
1,Alcoa Corp,AA,2021-05-04,37.374001,38.603001,37.009998,38.405998,6820821.0,NYSE


In [10]:
stocks_daily_ohlcv['OHLC_price'] = ( (0.1 * stocks_daily_ohlcv['Adj_Open']) + 
                                    (0.15 * stocks_daily_ohlcv['Adj_High']) + 
                                    (0.15 * stocks_daily_ohlcv['Adj_Low']) + 
                                    (0.6 * stocks_daily_ohlcv['Adj_Close']) ) 

In [11]:
for i in [1,2,3,5,10,20, 120,240, 300]:
    stocks_daily_ohlcv[f'Adj_Close_before_{i}d'] = stocks_daily_ohlcv.groupby("Ticker")['Adj_Close'].shift(i)

for i in [1,2,3,5,10,20, 120, 240, 300]:
    stocks_daily_ohlcv[f'Adj_Close_change_per_{i}d'] = (stocks_daily_ohlcv['Adj_Close'] - stocks_daily_ohlcv[f'Adj_Close_before_{i}d']) / stocks_daily_ohlcv[f'Adj_Close_before_{i}d']
    stocks_daily_ohlcv[f'Adj_Close_change_per_{i}d'].fillna(0,inplace=True)

stocks_daily_ohlcv['Adj_Close_change_per_1d_mean10'] = stocks_daily_ohlcv.groupby(['Ticker']).apply(lambda x: x[["Adj_Close_change_per_1d"]].rolling(window= 10).mean() ) #.reset_index(drop=True)

print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.tail(2) 


(755570, 29)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ,7.29150,7.26,7.39,7.32,7.45,7.45,8.01,8.90,6.52,6.37,0.004132,-0.013532,-0.004098,-0.021476,-0.021476,-0.089888,-0.180899,0.118098,0.144427,-0.002050
755569,Zymeworks Inc. Common Stock,ZYME,2023-08-09,7.29,7.725,7.29,7.70,514001.0,NASDAQ,7.60125,7.29,7.26,7.39,7.32,7.51,8.36,8.87,6.98,6.87,0.056241,0.060606,0.041949,0.051913,0.025300,-0.078947,-0.131905,0.103152,0.120815,0.002769


## Camarilla support resistance

In [12]:
"""
Monthly, 6 Monthly, Yearly - S1, R1
weekly - 1, 1p (S1, R1)
Daily - 1, 1p, 2 (S3, R3) 

Camarilla support resistance
Range = High - Low
Pivot Point (P) = (High + Low + Close) / 3
Support 1 (S1) = Close - Range * (1.1 / 12)
Support 3 (S3) = Close - Range * (1.1 / 4)
Resistance 1 (R1) = Close + Range * (1.1 / 12)
Resistance 3 (R3) = Close + Range * (1.1 / 4)
"""

def get_camarilla_DW_support_resistance_levels(DF, Adj_High, Adj_Low, Adj_Close):
    
    df = DF.copy()
    
    # Day 1, 1Prev levels: 
    df[f'{Adj_Close}_lag1'] = df.groupby('Ticker')[Adj_Close].shift(1)

    df[f'Cam_Range_1_days'] = df[Adj_High] - df[Adj_Low]
    df[f'Cam_PP_day_1'] = (df[Adj_High] + df[Adj_Low] + df[Adj_Close]) / 3
    df[f'Cam_R3_day_1'] = df[Adj_Close] + ( df[f'Cam_Range_1_days'] * (1.1 / 4) )
    df[f'Cam_S3_day_1'] = df[Adj_Close] - ( df[f'Cam_Range_1_days'] * (1.1 / 4) )
    
    df[f'Cam_Range_1P_days'] = df.groupby('Ticker')['Cam_Range_1_days'].shift(1)
    df[f'Cam_PP_day_1P'] = df.groupby('Ticker')['Cam_PP_day_1'].shift(1)
    df[f'Cam_R3_day_1P'] = df.groupby('Ticker')['Cam_R3_day_1'].shift(1)
    df[f'Cam_S3_day_1P'] = df.groupby('Ticker')['Cam_S3_day_1'].shift(1)


    for var in ['Cam_PP_day_1', 'Cam_R3_day_1', 'Cam_S3_day_1']:
        df[f'{var}_per'] = 100 * (df[var] - df[Adj_Close]) / df[Adj_Close]
        
    for var in ['Cam_PP_day_1P', 'Cam_R3_day_1P', 'Cam_S3_day_1P']:
        df[f'{var}_per'] = 100 * (df[var] - df[f'{Adj_Close}_lag1']) / df[f'{Adj_Close}_lag1']

    df.drop(columns=[f'{Adj_Close}_lag1' ], inplace=True)
    
    # Day 5, 5Prev i.e. Weekly levels: 
    df[f'{Adj_High}_5_days'] = df.groupby('Ticker')[Adj_High].rolling(5).max().reset_index(drop=True)
    df[f'{Adj_Low}_5_days'] = df.groupby('Ticker')[Adj_Low].rolling(5).min().reset_index(drop=True)
    
    df[f'{Adj_Close}_lag5'] = df.groupby('Ticker')[Adj_Close].shift(5)
    
    df[f'Cam_Range_5_days'] = df[f'{Adj_High}_5_days'] - df[f'{Adj_Low}_5_days']
    df[f'Cam_PP_day_5'] = (df[f'{Adj_High}_5_days'] + df[f'{Adj_Low}_5_days'] + df[Adj_Close]) / 3
    df[f'Cam_R1_day_5'] = df[Adj_Close] + ( df[f'Cam_Range_5_days'] * (1.1 / 12) )
    df[f'Cam_S1_day_5'] = df[Adj_Close] - ( df[f'Cam_Range_5_days'] * (1.1 / 12) )
    
    df[f'Cam_Range_5P_days'] = df.groupby('Ticker')['Cam_Range_5_days'].shift(5)
    df[f'Cam_PP_day_5P'] = df.groupby('Ticker')['Cam_PP_day_5'].shift(5)
    df[f'Cam_R1_day_5P'] = df.groupby('Ticker')['Cam_R1_day_5'].shift(5)
    df[f'Cam_S1_day_5P'] = df.groupby('Ticker')['Cam_S1_day_5'].shift(5)


    for var in ['Cam_PP_day_5', 'Cam_R1_day_5', 'Cam_S1_day_5']:
        df[f'{var}_per'] = 100 * (df[var] - df[Adj_Close]) / df[Adj_Close]
        
    for var in ['Cam_PP_day_5P', 'Cam_R1_day_5P', 'Cam_S1_day_5P']:
        df[f'{var}_per'] = 100 * (df[var] - df[f'{Adj_Close}_lag5']) / df[f'{Adj_Close}_lag5']

    df.drop(columns=[f'{Adj_High}_5_days', f'{Adj_Low}_5_days', f'{Adj_Close}_lag5'
                     ], inplace=True)
    
    return df

stocks_daily_ohlcv = get_camarilla_DW_support_resistance_levels(stocks_daily_ohlcv, 'Adj_High', 'Adj_Low', 'Adj_Close')
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.head(2) 

(755570, 57)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per
0,Alcoa Corp,AA,2021-05-03,36.548000,37.408001,35.437000,36.971001,4738228.5,NYSE,36.764153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.971001,36.605335,37.513027,36.428974,NaN,NaN,NaN,NaN,-0.989060,1.466086,-1.466086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alcoa Corp,AA,2021-05-04,37.374001,38.603001,37.009998,38.405998,6820821.0,NYSE,38.122948,36.971001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.593002,38.006332,38.844074,37.967922,1.971001,36.605335,37.513027,36.428974,-1.040634,1.140645,-1.140645,-0.98906,1.466086,-1.466086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def get_camarilla_group_support_resistance_levels(DF, Adj_High, Adj_Low, Adj_Close, days=[2,20,120,240]):
    
    df = DF.copy()
    
    for index, i in enumerate(days):
        
        df[f'{Adj_High}_{i}_days'] = df.groupby('Ticker')[Adj_High].rolling(i).max().reset_index(drop=True)
        df[f'{Adj_Low}_{i}_days'] = df.groupby('Ticker')[Adj_Low].rolling(i).min().reset_index(drop=True)
        
        df[f'Cam_Range_{i}_days'] = df[f'{Adj_High}_{i}_days'] - df[f'{Adj_Low}_{i}_days']
        df[f'Cam_PP_day_{i}'] = (df[f'{Adj_High}_{i}_days'] + df[f'{Adj_Low}_{i}_days'] + df[Adj_Close]) / 3
        df[f'Cam_R1_day_{i}'] = df[Adj_Close] + ( df[f'Cam_Range_{i}_days'] * (1.1 / 12) )
        df[f'Cam_S1_day_{i}'] = df[Adj_Close] - ( df[f'Cam_Range_{i}_days'] * (1.1 / 12) )
        
        for var in [f'{Adj_High}_{i}_days', f'{Adj_Low}_{i}_days', f'Cam_PP_day_{i}', f'Cam_R1_day_{i}', f'Cam_S1_day_{i}']:
            df[f'{var}_per'] = 100 * (df[var] - df[Adj_Close]) / df[Adj_Close]
            
        df.drop(columns=[f'{Adj_High}_{i}_days', f'{Adj_Low}_{i}_days', f'Cam_Range_{i}_days', f'Cam_PP_day_{i}',
                         f'Cam_R1_day_{i}', f'Cam_S1_day_{i}'
                        ], inplace=True)
             
    return df
stocks_daily_ohlcv = get_camarilla_group_support_resistance_levels(stocks_daily_ohlcv, 'Adj_High', 'Adj_Low', 'Adj_Close', days=[2,20,120,240])
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.head(2) 

(755570, 77)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per
0,Alcoa Corp,AA,2021-05-03,36.548000,37.408001,35.437000,36.971001,4738228.5,NYSE,36.764153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.971001,36.605335,37.513027,36.428974,NaN,NaN,NaN,NaN,-0.989060,1.466086,-1.466086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alcoa Corp,AA,2021-05-04,37.374001,38.603001,37.009998,38.405998,6820821.0,NYSE,38.122948,36.971001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.593002,38.006332,38.844074,37.967922,1.971001,36.605335,37.513027,36.428974,-1.040634,1.140645,-1.140645,-0.98906,1.466086,-1.466086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.512947,-7.730558,-2.40587,0.755655,-0.755655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# creating Graph features
print(stocks_daily_ohlcv.shape) 

stocks_daily_ohlcv = hfs.get_RSI(stocks_daily_ohlcv, window_size = 9, close_col = "Adj_Close") 
stocks_daily_ohlcv = hfs.get_RSI(stocks_daily_ohlcv, window_size = 14, close_col = "Adj_Close") 
stocks_daily_ohlcv = hfs.get_RSI(stocks_daily_ohlcv, window_size = 20, close_col = "Adj_Close") 

stocks_daily_ohlcv = hfs.get_volatility(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 126)
stocks_daily_ohlcv = hfs.get_max_drop(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 126)

print(stocks_daily_ohlcv.shape) 

stocks_daily_ohlcv = hfs.get_volatility(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 20)
stocks_daily_ohlcv = hfs.get_max_drop(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 20)

print(stocks_daily_ohlcv.shape) 

stocks_daily_ohlcv = hfs.get_TR_ATR(stocks_daily_ohlcv, window_size = 3 ,high_col = "Adj_High", low_col = "Adj_Low", close_col = "Adj_Close")

stocks_daily_ohlcv = hfs.get_volatility(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 5)
stocks_daily_ohlcv = hfs.get_volatility(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 3)
stocks_daily_ohlcv = hfs.get_volatility(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 2)

stocks_daily_ohlcv = hfs.get_max_drop(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 5)
stocks_daily_ohlcv = hfs.get_max_drop(stocks_daily_ohlcv, close_col = "Adj_Close", window_size = 3)


stocks_daily_ohlcv = hfs.get_QM_ADR(stocks_daily_ohlcv, window_size = 5 ,high_col = "Adj_High", low_col = "Adj_Low", close_col = "Adj_Close")
stocks_daily_ohlcv = hfs.get_QM_ADR(stocks_daily_ohlcv, window_size = 20,high_col = "Adj_High", low_col = "Adj_Low", close_col = "Adj_Close")

stocks_daily_ohlcv = hfs.get_BollBnd(stocks_daily_ohlcv, window_size = 20, x_std_dev=2, close_col = "Adj_Close")
stocks_daily_ohlcv = hfs.get_BollBnd(stocks_daily_ohlcv, window_size = 20, x_std_dev=3, close_col = "Adj_Close")

print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 

(755570, 77)
(755570, 82)
(755570, 84)
(755570, 97)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ,7.292,7.26,7.39,7.32,7.45,7.45,8.01,8.90,6.52,6.37,0.004,-0.014,-0.004,-0.021,-0.021,-0.090,-0.181,0.118,0.144,-0.002,0.290,7.307,7.37,7.21,0.338,7.253,7.353,7.167,0.229,1.094,-1.094,-0.101,1.280,-1.280,0.550,7.333,7.340,7.240,0.42,7.450,7.488,7.411,0.594,0.692,-0.692,0.000,0.517,-0.517,2.332,-2.881,-0.183,0.478,-0.478,17.970,-2.881,5.030,1.911,-1.911,41.975,-2.881,13.032,4.112,-4.112,48.148,-43.621,1.509,8.412,-8.412,36.031,37.482,39.281,0.332,0.339,0.107,0.135,0.290,0.329,0.028,0.025,0.022,0.018,0.0,4.615,4.137,8.291,6.939,8.629,6.601
755569,Zymeworks Inc. Common Stock,ZYME,2023-08-09,7.29,7.725,7.29,7.70,514001.0,NASDAQ,7.601,7.29,7.26,7.39,7.32,7.51,8.36,8.87,6.98,6.87,0.056,0.061,0.042,0.052,0.025,-0.079,-0.132,0.103,0.121,0.003,0.435,7.572,7.82,7.58,0.290,7.307,7.370,7.210,-1.667,1.554,-1.554,0.229,1.094,-1.094,0.645,7.502,7.759,7.641,0.49,7.383,7.365,7.275,-2.576,0.768,-0.768,0.865,0.614,-0.614,0.325,-6.883,-2.186,0.661,-0.661,11.688,-8.052,1.212,1.810,-1.810,34.416,-8.052,8.788,3.893,-3.893,40.260,-46.623,-2.121,7.964,-7.964,54.582,49.010,46.795,0.337,0.339,0.113,0.096,0.435,0.354,0.062,0.066,0.052,0.018,0.0,4.804,4.232,8.168,6.996,8.461,6.703


### OHLCV standard Features

In [15]:
print(stocks_daily_ohlcv.shape) 
now = datetime.now() 

stocks_daily_ohlcv = hfs.create_set_of_standard_features_ana(stocks_daily_ohlcv, "Ticker", "Adj_Close")
print((datetime.now() - now).total_seconds()) 

print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 


(755570, 97)
0.435874
(755570, 112)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ,7.292,7.26,7.39,7.32,7.45,7.45,8.01,8.90,6.52,6.37,0.004,-0.014,-0.004,-0.021,-0.021,-0.090,-0.181,0.118,0.144,-0.002,0.290,7.307,7.37,7.21,0.338,7.253,7.353,7.167,0.229,1.094,-1.094,-0.101,1.280,-1.280,0.550,7.333,7.340,7.240,0.42,7.450,7.488,7.411,0.594,0.692,-0.692,0.000,0.517,-0.517,2.332,-2.881,-0.183,0.478,-0.478,17.970,-2.881,5.030,1.911,-1.911,41.975,-2.881,13.032,4.112,-4.112,48.148,-43.621,1.509,8.412,-8.412,36.031,37.482,39.281,0.332,0.339,0.107,0.135,0.290,0.329,0.028,0.025,0.022,0.018,0.0,4.615,4.137,8.291,6.939,8.629,6.601,7.26,8.328,8.653,8.587,8.611,8.085,7.963,7.773,7.615,7.460,7.366,7.355,7.316,7.313,7.275
755569,Zymeworks Inc. Common Stock,ZYME,2023-08-09,7.29,7.725,7.29,7.70,514001.0,NASDAQ,7.601,7.29,7.26,7.39,7.32,7.51,8.36,8.87,6.98,6.87,0.056,0.061,0.042,0.052,0.025,-0.079,-0.132,0.103,0.121,0.003,0.435,7.572,7.82,7.58,0.290,7.307,7.370,7.210,-1.667,1.554,-1.554,0.229,1.094,-1.094,0.645,7.502,7.759,7.641,0.49,7.383,7.365,7.275,-2.576,0.768,-0.768,0.865,0.614,-0.614,0.325,-6.883,-2.186,0.661,-0.661,11.688,-8.052,1.212,1.810,-1.810,34.416,-8.052,8.788,3.893,-3.893,40.260,-46.623,-2.121,7.964,-7.964,54.582,49.010,46.795,0.337,0.339,0.113,0.096,0.435,0.354,0.062,0.066,0.052,0.018,0.0,4.804,4.232,8.168,6.996,8.461,6.703,7.29,8.335,8.654,8.581,8.599,8.074,7.947,7.739,7.582,7.449,7.392,7.374,7.392,7.417,7.495


In [16]:
print(stocks_daily_ohlcv.shape) 

(755570, 112)


In [17]:
stocks_daily_ohlcv['Adj_Close_lag1'] = stocks_daily_ohlcv.groupby('Ticker')['Adj_Close'].shift(1)
stocks_daily_ohlcv['D1_Close_volatility_percnt'] = (stocks_daily_ohlcv['Adj_High'] - stocks_daily_ohlcv['Adj_Low']) / stocks_daily_ohlcv['Adj_Close_lag1']
stocks_daily_ohlcv[f'D1_Close_volatility_percnt'].fillna(0,inplace=True)
stocks_daily_ohlcv.head(2) 

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt
0,Alcoa Corp,AA,2021-05-03,36.548000,37.408001,35.437000,36.971001,4738228.5,NYSE,36.764000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.971,36.605000,37.513000,36.429001,NaN,NaN,NaN,NaN,-0.989,1.466,-1.466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,Alcoa Corp,AA,2021-05-04,37.374001,38.603001,37.009998,38.405998,6820821.0,NYSE,38.123001,36.971001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.593,38.006001,38.844002,37.967999,1.971,36.605,37.513,36.429001,-1.041,1.141,-1.141,-0.989,1.466,-1.466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513,-7.731,-2.406,0.756,-0.756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.971001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.688,0.043088


In [18]:
for i in [1,2,3,5,10,20]:
    stocks_daily_ohlcv[f'D1_Close_volatility_percnt_before_{i}d'] = stocks_daily_ohlcv.groupby("Ticker")['D1_Close_volatility_percnt'].shift(i)

for i in [1,2,3,5,10,20]:
    stocks_daily_ohlcv[f'price_voilatility_change_last_{i}d'] = (stocks_daily_ohlcv['D1_Close_volatility_percnt'] - stocks_daily_ohlcv[f'D1_Close_volatility_percnt_before_{i}d']) / stocks_daily_ohlcv[f'D1_Close_volatility_percnt_before_{i}d']
    stocks_daily_ohlcv[f'price_voilatility_change_last_{i}d'].fillna(0, inplace=True)
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.tail(2) 

(755570, 125)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ,7.292,7.26,7.39,7.32,7.45,7.45,8.01,8.90,6.52,6.37,0.004,-0.014,-0.004,-0.021,-0.021,-0.090,-0.181,0.118,0.144,-0.002,0.290,7.307,7.37,7.21,0.338,7.253,7.353,7.167,0.229,1.094,-1.094,-0.101,1.280,-1.280,0.550,7.333,7.340,7.240,0.42,7.450,7.488,7.411,0.594,0.692,-0.692,0.000,0.517,-0.517,2.332,-2.881,-0.183,0.478,-0.478,17.970,-2.881,5.030,1.911,-1.911,41.975,-2.881,13.032,4.112,-4.112,48.148,-43.621,1.509,8.412,-8.412,36.031,37.482,39.281,0.332,0.339,0.107,0.135,0.290,0.329,0.028,0.025,0.022,0.018,0.0,4.615,4.137,8.291,6.939,8.629,6.601,7.26,8.328,8.653,8.587,8.611,8.085,7.963,7.773,7.615,7.460,7.366,7.355,7.316,7.313,7.275,0.039945,0.045738,0.049180,0.04235,0.025469,0.041272,0.037240,-0.126649,-0.187787,-0.056786,0.568362,-0.032156,0.072621
755569,Zymeworks Inc. Common Stock,ZYME,2023-08-09,7.29,7.725,7.29,7.70,514001.0,NASDAQ,7.601,7.29,7.26,7.39,7.32,7.51,8.36,8.87,6.98,6.87,0.056,0.061,0.042,0.052,0.025,-0.079,-0.132,0.103,0.121,0.003,0.435,7.572,7.82,7.58,0.290,7.307,7.370,7.210,-1.667,1.554,-1.554,0.229,1.094,-1.094,0.645,7.502,7.759,7.641,0.49,7.383,7.365,7.275,-2.576,0.768,-0.768,0.865,0.614,-0.614,0.325,-6.883,-2.186,0.661,-0.661,11.688,-8.052,1.212,1.810,-1.810,34.416,-8.052,8.788,3.893,-3.893,40.260,-46.623,-2.121,7.964,-7.964,54.582,49.010,46.795,0.337,0.339,0.113,0.096,0.435,0.354,0.062,0.066,0.052,0.018,0.0,4.804,4.232,8.168,6.996,8.461,6.703,7.29,8.335,8.654,8.581,8.599,8.074,7.947,7.739,7.582,7.449,7.392,7.374,7.392,7.417,7.495,0.059671,0.039945,0.045738,0.04918,0.048322,0.030738,0.041198,0.493827,0.304635,0.213305,0.234853,0.941254,0.448373


In [19]:
stocks_daily_ohlcv['gap_percent'] = 100 *( (stocks_daily_ohlcv['Adj_Open'] - stocks_daily_ohlcv['Adj_Close_before_1d']) / stocks_daily_ohlcv['Adj_Close_before_1d'] )


In [20]:
stocks_daily_ohlcv['gap_5day_max'] = stocks_daily_ohlcv['gap_percent'].rolling(5).max()
stocks_daily_ohlcv['gap_5day_min'] = stocks_daily_ohlcv['gap_percent'].rolling(5).min()

In [21]:
stocks_daily_ohlcv['High_52_weeks'] = stocks_daily_ohlcv["Adj_High"].rolling(52*5).max()
stocks_daily_ohlcv['Low_52_weeks'] = stocks_daily_ohlcv["Adj_Low"].rolling(52*5).min()

stocks_daily_ohlcv['High_26_weeks'] = stocks_daily_ohlcv["Adj_High"].rolling(26*5).max()
stocks_daily_ohlcv['Low_26_weeks'] = stocks_daily_ohlcv["Adj_Low"].rolling(26*5).min()

stocks_daily_ohlcv['High_4_weeks'] = stocks_daily_ohlcv["Adj_High"].rolling(4*5).max()
stocks_daily_ohlcv['Low_4_weeks'] = stocks_daily_ohlcv["Adj_Low"].rolling(4*5).min()

stocks_daily_ohlcv['High_1_week'] = stocks_daily_ohlcv["Adj_High"].rolling(5).max()
stocks_daily_ohlcv['Low_1_week'] = stocks_daily_ohlcv["Adj_Low"].rolling(5).min() 

In [22]:
stocks_daily_ohlcv['body_percent'] = abs((stocks_daily_ohlcv['Adj_Close'] - stocks_daily_ohlcv['Adj_Open'])) / \
                                     (stocks_daily_ohlcv['Adj_High'] - stocks_daily_ohlcv['Adj_Low'])

stocks_daily_ohlcv['body_per_5d_mean'] = stocks_daily_ohlcv['body_percent'].rolling(5).mean()
stocks_daily_ohlcv['body_per_20d_mean'] = stocks_daily_ohlcv['body_percent'].rolling(20).mean()
stocks_daily_ohlcv['body_per_3month_mean'] = stocks_daily_ohlcv['body_percent'].rolling(60).mean()

In [23]:
stocks_daily_ohlcv.tail(2)

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ,7.292,7.26,7.39,7.32,7.45,7.45,8.01,8.90,6.52,6.37,0.004,-0.014,-0.004,-0.021,-0.021,-0.090,-0.181,0.118,0.144,-0.002,0.290,7.307,7.37,7.21,0.338,7.253,7.353,7.167,0.229,1.094,-1.094,-0.101,1.280,-1.280,0.550,7.333,7.340,7.240,0.42,7.450,7.488,7.411,0.594,0.692,-0.692,0.000,0.517,-0.517,2.332,-2.881,-0.183,0.478,-0.478,17.970,-2.881,5.030,1.911,-1.911,41.975,-2.881,13.032,4.112,-4.112,48.148,-43.621,1.509,8.412,-8.412,36.031,37.482,39.281,0.332,0.339,0.107,0.135,0.290,0.329,0.028,0.025,0.022,0.018,0.0,4.615,4.137,8.291,6.939,8.629,6.601,7.26,8.328,8.653,8.587,8.611,8.085,7.963,7.773,7.615,7.460,7.366,7.355,7.316,7.313,7.275,0.039945,0.045738,0.049180,0.04235,0.025469,0.041272,0.037240,-0.126649,-0.187787,-0.056786,0.568362,-0.032156,0.072621,-0.413226,1.092895,-1.342281,10.8,4.11,10.35,7.08,8.6,7.08,7.630,7.08,0.206896,0.225109,0.361111,0.426185
755569,Zymeworks Inc. Common Stock,ZYME,2023-08-09,7.29,7.725,7.29,7.70,514001.0,NASDAQ,7.601,7.29,7.26,7.39,7.32,7.51,8.36,8.87,6.98,6.87,0.056,0.061,0.042,0.052,0.025,-0.079,-0.132,0.103,0.121,0.003,0.435,7.572,7.82,7.58,0.290,7.307,7.370,7.210,-1.667,1.554,-1.554,0.229,1.094,-1.094,0.645,7.502,7.759,7.641,0.49,7.383,7.365,7.275,-2.576,0.768,-0.768,0.865,0.614,-0.614,0.325,-6.883,-2.186,0.661,-0.661,11.688,-8.052,1.212,1.810,-1.810,34.416,-8.052,8.788,3.893,-3.893,40.260,-46.623,-2.121,7.964,-7.964,54.582,49.010,46.795,0.337,0.339,0.113,0.096

In [24]:
stocks_daily_ohlcv['Long_MA20_crossed_MA200_in_5d'] = ((stocks_daily_ohlcv['Adj_Close_MA20'] > stocks_daily_ohlcv['Adj_Close_MA200']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA20'].rolling(window=5).min() < stocks_daily_ohlcv['Adj_Close_MA200'].rolling(window=5).max())
                                 )


stocks_daily_ohlcv['Short_MA20_crossed_MA200_in_5d'] = ((stocks_daily_ohlcv['Adj_Close_MA20'] < stocks_daily_ohlcv['Adj_Close_MA200']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA20'].rolling(window=5).max() > stocks_daily_ohlcv['Adj_Close_MA200'].rolling(window=5).min())
                                 )


In [25]:
stocks_daily_ohlcv['Long_MA10_crossed_MA50_in_5d'] = ((stocks_daily_ohlcv['Adj_Close_MA10'] > stocks_daily_ohlcv['Adj_Close_MA50']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA10'].rolling(window=5).min() < stocks_daily_ohlcv['Adj_Close_MA50'].rolling(window=5).max())
                                 )


stocks_daily_ohlcv['Short_MA10_crossed_MA50_in_2d'] = ((stocks_daily_ohlcv['Adj_Close_MA10'] < stocks_daily_ohlcv['Adj_Close_MA50']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA10'].rolling(window=5).max() > stocks_daily_ohlcv['Adj_Close_MA50'].rolling(window=5).min())
                                 )

In [26]:
stocks_daily_ohlcv['Long_MA20_crossed_MA200_in_2d'] = ((stocks_daily_ohlcv['Adj_Close_MA20'] > stocks_daily_ohlcv['Adj_Close_MA200']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA20'].rolling(window=2).min() < stocks_daily_ohlcv['Adj_Close_MA200'].rolling(window=2).max())
                                 )


stocks_daily_ohlcv['Short_MA20_crossed_MA200_in_2d'] = ((stocks_daily_ohlcv['Adj_Close_MA20'] < stocks_daily_ohlcv['Adj_Close_MA200']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA20'].rolling(window=2).max() > stocks_daily_ohlcv['Adj_Close_MA200'].rolling(window=2).min())
                                 )


In [27]:
stocks_daily_ohlcv['Long_MA10_crossed_MA50_in_2d'] = ((stocks_daily_ohlcv['Adj_Close_MA10'] > stocks_daily_ohlcv['Adj_Close_MA50']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA10'].rolling(window=2).min() < stocks_daily_ohlcv['Adj_Close_MA50'].rolling(window=2).max())
                                 )


stocks_daily_ohlcv['Short_MA10_crossed_MA50_in_2d'] = ((stocks_daily_ohlcv['Adj_Close_MA10'] < stocks_daily_ohlcv['Adj_Close_MA50']) & 
                                  (stocks_daily_ohlcv['Adj_Close_MA10'].rolling(window=2).max() > stocks_daily_ohlcv['Adj_Close_MA50'].rolling(window=2).min())
                                 )

In [28]:
base_feature = "Adj_Close"
stocks_daily_ohlcv = hfs.get_EMA(stocks_daily_ohlcv, base_feature, window_size_list=[5,9,10,20]) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 

(755570, 151)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ,7.292,7.26,7.39,7.32,7.45,7.45,8.01,8.90,6.52,6.37,0.004,-0.014,-0.004,-0.021,-0.021,-0.090,-0.181,0.118,0.144,-0.002,0.290,7.307,7.37,7.21,0.338,7.253,7.353,7.167,0.229,1.094,-1.094,-0.101,1.280,-1.280,0.550,7.333,7.340,7.240,0.42,7.450,7.488,7.411,0.594,0.692,-0.692,0.000,0.517,-0.517,2.332,-2.881,-0.183,0.478,-0.478,17.970,-2.881,5.030,1.911,-1.911,41.975,-2.881,13.032,4.112,-4.112,48.148,-43.621,1.509,8.412,-8.412,36.031,37.482,39.281,0.332,0.339,0.107,0.135,0.290,0.329,0.028,0.025,0.022,0.018,0.0,4.615,4.137,8.291,6.939,8.629,6.601,7.26,8.328,8.653,8.587,8.611,8.085,7.963,7.773,7.615,7.460,7.366,7.355,7.316,7.313,7.275,0.039945,0.045738,0.049180,0.04235,0.025469,0.041272,0.037240,-0.126649,-0.187787,-0.056786,0.568362,-0.032156,0.072621,-0.413226,1.092895,-1.342281,10.8,4.11,10.35,7.08,8.6,7.08,7.630,7.08,0.206896,0.225109,0.361111,0.426185,False,False,False,False,False,False,False,7.322408,7.379854,7.397595,7.589101
755569,Zymeworks Inc. Common Stock,ZYME,2023-08-09,7.29,7.725,7.29,7.70,514001.0,NASDAQ,7.601,7.29,7.26,7.39,7.32,7.51,8.36,8.87,6.98,6.87,0.056,0.061,0.042,0.052,0.025,-0.079,-0.132,0.103,0.121,0.003,0.435,7.572,7.82,7.58,0.290,7.307,7

In [29]:


stocks_daily_ohlcv['Low_52_weeks'] = stocks_daily_ohlcv["Adj_Low"].rolling(52*5).min()
stocks_daily_ohlcv['ATR'] = stocks_daily_ohlcv['Adj_High'] - stocks_daily_ohlcv['Adj_Low']

stocks_daily_ohlcv['KCupper'] = stocks_daily_ohlcv['Adj_Close_MA10'] + 1.5*stocks_daily_ohlcv['ATR'].rolling(window=10).mean() 
stocks_daily_ohlcv['KClower'] = stocks_daily_ohlcv['Adj_Close_MA10'] - 1.5*stocks_daily_ohlcv['ATR'].rolling(window=10).mean() 


In [30]:
print(stocks_daily_ohlcv.info()) 
stocks_daily_ohlcv = hfs.convert_floats_to_float32(stocks_daily_ohlcv) 
stocks_daily_ohlcv.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755570 entries, 0 to 755569
Columns: 154 entries, company_Name to KClower
dtypes: bool(7), datetime64[ns](1), float32(56), float64(87), object(3)
memory usage: 691.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755570 entries, 0 to 755569
Columns: 154 entries, company_Name to KClower
dtypes: bool(7), datetime64[ns](1), float32(143), object(3)
memory usage: 440.3+ MB


In [31]:
stocks_daily_ohlcv['High_Open_diff'] = stocks_daily_ohlcv['Adj_High'] - stocks_daily_ohlcv['Adj_Open'] 
stocks_daily_ohlcv['Low_Open_diff'] = stocks_daily_ohlcv['Adj_Low'] - stocks_daily_ohlcv['Adj_Open'] 
stocks_daily_ohlcv['High_Low_diff'] = stocks_daily_ohlcv['Adj_High'] - stocks_daily_ohlcv['Adj_Low'] 


stocks_daily_ohlcv['High_Open_diff_5d_max'] = stocks_daily_ohlcv['High_Open_diff'].rolling(5).max()
stocks_daily_ohlcv['High_Open_diff_5d_min'] = stocks_daily_ohlcv['High_Open_diff'].rolling(5).min()

stocks_daily_ohlcv['Low_Open_diff_5d_max'] = stocks_daily_ohlcv['Low_Open_diff'].rolling(5).max()
stocks_daily_ohlcv['Low_Open_diff_5d_min'] = stocks_daily_ohlcv['Low_Open_diff'].rolling(5).min()

stocks_daily_ohlcv['High_Low_diff_5d_max'] = stocks_daily_ohlcv['High_Low_diff'].rolling(5).max()
stocks_daily_ohlcv['High_Low_diff_5d_min'] = stocks_daily_ohlcv['High_Low_diff'].rolling(5).min()


stocks_daily_ohlcv['straight_down'] = np.abs(stocks_daily_ohlcv['Low_Open_diff']) >= 0.85 * stocks_daily_ohlcv['High_Low_diff']
stocks_daily_ohlcv['straight_up'] = np.abs(stocks_daily_ohlcv['High_Open_diff']) >= 0.85 * stocks_daily_ohlcv['High_Low_diff']

stocks_daily_ohlcv['straight_down_per'] = np.where( stocks_daily_ohlcv['High_Low_diff'] ==0,
                                                    0,
                                                    np.abs(stocks_daily_ohlcv['Low_Open_diff'] / stocks_daily_ohlcv['High_Low_diff']))

stocks_daily_ohlcv['straight_up_per'] =  np.where(  stocks_daily_ohlcv['High_Low_diff']==0,
                                                    0,
                                                    np.abs(stocks_daily_ohlcv['High_Open_diff'] / stocks_daily_ohlcv['High_Low_diff'])
                                                 )


stocks_daily_ohlcv['Close_Low_diff'] = stocks_daily_ohlcv['Adj_Close'] - stocks_daily_ohlcv['Adj_Low'] 
stocks_daily_ohlcv['Close_Open_diff'] = stocks_daily_ohlcv['Adj_Close'] - stocks_daily_ohlcv['Adj_Open'] 

stocks_daily_ohlcv['straight_up_close_per'] =   np.where( stocks_daily_ohlcv['Close_Low_diff']==0,
                                                          0,
                                                          np.abs(stocks_daily_ohlcv['Close_Open_diff'] / stocks_daily_ohlcv['Close_Low_diff'])
                                                       )


# Volume Features

In [32]:
stocks_daily_ohlcv = hfs.create_set_of_standard_Volume_features(stocks_daily_ohlcv, "Ticker", "Adj_Vol") 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 

(755570, 181)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol
755568,Zymeworks Inc. Common Stock,ZYME,2023-08-08,7.23,7.460,7.17,7.29,394500.0,NASDAQ,7.292,7.26,7.39,7.32,7.45,7.45,8.01,8.90,6.52,6.37,0.004,-0.014,-0.004,-0.021,-0.021,-0.090,-0.181,0.118,0.144,-0.002,0.290,7.307,7.37,7.21,0.338,7.253,7.353,7.167,0.229,1.094,-1.094,-0.101,1.280,-1.280,0.550,7.333,7.340,7.240,0.42,7.450,7.488,7.411,0.594,0.692,-0.692,0.000,0.517,-0.517,2.332,-2.881,-0.183,0.478,-0.478,17.969999,-2.881,5.030,1.911,-1.911,41.974998,-2.881,13.032,4.112,-4.112,48.147999,-43.620998,1.509,8.412,-8.412,36.030998,37.481998,39.280998,0.332,0.339,0.107,0.135,0.290,0.329,0.028,0.025,0.022,0.018,0.0,4.615,4.137,8.291,6.939,8.629,6.601,7.26,8.328,8.653,8.587,8.611,8.085,7.963,7.773,7.615,7.460,7.366,7.355,7.316,7.313,7.275,0.04,0.046,0.049,0.042,0.025,0.041,0.037,-0.127,-0.188,-0.057,0.56

In [33]:
for i in [1,2,3,5,10,20]:
    stocks_daily_ohlcv[f'Adj_Vol_before_{i}d'] = stocks_daily_ohlcv.groupby("Ticker")['Adj_Vol'].shift(i)
    stocks_daily_ohlcv[f'Adj_Vol_before_{i}d'].fillna(0, inplace=True) 

for i in [1,2,3,5,10,20]:
    stocks_daily_ohlcv[f'Adj_Vol_change_per_{i}d'] = (stocks_daily_ohlcv['Adj_Vol'] - stocks_daily_ohlcv[f'Adj_Vol_before_{i}d']) / stocks_daily_ohlcv[f'Adj_Vol_before_{i}d']
    stocks_daily_ohlcv[f'Adj_Vol_change_per_{i}d'].fillna(0, inplace=True) 

In [34]:
now = datetime.now() 

stocks_daily_ohlcv = hfs.get_VWAP(stocks_daily_ohlcv, "Ticker", "Adj_Vol", "Adj_Close", 5) 
stocks_daily_ohlcv = hfs.get_VWAP(stocks_daily_ohlcv, "Ticker", "Adj_Vol", "Adj_Close", 3) 
stocks_daily_ohlcv = hfs.get_VWAP(stocks_daily_ohlcv, "Ticker", "Adj_Vol", "Adj_Close", 2) 
print("close done") 
print((datetime.now() - now).total_seconds()) 

print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(3) 


close done
1.596484
(755570, 196)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2
755567,Zymeworks Inc. Common Stock,ZYME,2023-08-07,7.38,7.418,7.08,7.26,430800.0,NASDAQ,7.269,7.39,7.32,7.32,7.46,7.39,8.19,8.92,6.40,5.92,-0.018,-0.008,-0.008,-0.027,-0.018,-0.114,-0.186,0.134,0.226,-0.002,0.338,7.253,7.353,7.167,0.360,7.430,7.489,7.291,-0.101,1.280,-1.280,0.541,1.340,-1.340,0.550,7.323,7.310,7.210,0.435,7.458,7.500,7.420,0.872,0.694,-0.694,-0.022,0.535,-0.535,5.096,-2.479,0.872,0.694,-0.694,18.457001,-2.479,5.326,1.919,-1.919,42.562000,-2.479,13.361,4.129,-4.129,48.759998,-43.388000,1.791,8.

In [35]:
stocks_daily_ohlcv['last_5_days_gr_status'] = list(zip(stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(0), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(1), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(2), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(3), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(4)
                                         ))

stocks_daily_ohlcv['last_10_days_gr_status'] = list(zip(stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(0), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(1), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(2), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(3), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(4),
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(5), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(6), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(7), 
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(8),
                                          stocks_daily_ohlcv.groupby("Ticker")['Adj_Close_change_per_1d'].shift(9)
                                         ))

stocks_daily_ohlcv['5_day_negatives'] = stocks_daily_ohlcv['last_5_days_gr_status'].apply(lambda x: len(list(filter(lambda i: i < 0, x))) )
stocks_daily_ohlcv['10_day_negatives'] = stocks_daily_ohlcv['last_10_days_gr_status'].apply(lambda x: len(list(filter(lambda i: i < 0, x))) )

stocks_daily_ohlcv.drop(columns=['last_5_days_gr_status','last_10_days_gr_status'], inplace=True ) 

In [36]:

stocks_daily_ohlcv[f'Adj_Vol_1d'] = stocks_daily_ohlcv.groupby("Ticker")['Adj_Vol'].shift(1)

for i in [5,20]:
    # analytics_df[f'Adj_Vol_EMA_prev_{i}d'] = analytics_df['Adj_Vol_1d'].ewm(span=i).mean() #
    stocks_daily_ohlcv[f'Adj_Vol_EMA_prev_{i}d'] = stocks_daily_ohlcv['Adj_Vol_1d'].rolling(i).mean().round(decimals=3)
    
    
stocks_daily_ohlcv['EMA_prev_20d_DIV_Adj_Vol'] = stocks_daily_ohlcv['Adj_Vol_EMA_prev_20d'] / stocks_daily_ohlcv['Adj_Vol']
stocks_daily_ohlcv['EMA_prev_5d_DIV_Adj_Vol'] = stocks_daily_ohlcv['Adj_Vol_EMA_prev_5d'] / stocks_daily_ohlcv['Adj_Vol']

stocks_daily_ohlcv['Opening_Gap_Percent'] = (stocks_daily_ohlcv['Adj_Open'] - stocks_daily_ohlcv['Adj_Close_before_1d']) /  stocks_daily_ohlcv['Adj_Close_before_1d']

stocks_daily_ohlcv['day_Open_high_per'] = stocks_daily_ohlcv['High_Open_diff'] / stocks_daily_ohlcv['Adj_Open']
stocks_daily_ohlcv['day_Open_low_per'] = stocks_daily_ohlcv['Low_Open_diff'] / stocks_daily_ohlcv['Adj_Open']
stocks_daily_ohlcv['day_Open_close_per'] = stocks_daily_ohlcv['Close_Open_diff'] / stocks_daily_ohlcv['Adj_Open']

stocks_daily_ohlcv['ATR_3_DIV_OHLC_price'] = stocks_daily_ohlcv['ATR_3'] / stocks_daily_ohlcv['OHLC_price']
                         
stocks_daily_ohlcv['Cam_R1_day_5P_DIV_Cam_R1_day_5'] = stocks_daily_ohlcv['Cam_R1_day_5P'] / stocks_daily_ohlcv['Cam_R1_day_5'] 
stocks_daily_ohlcv['Cam_S1_day_5P_DIV_Cam_S1_day_5'] = stocks_daily_ohlcv['Cam_S1_day_5P'] / stocks_daily_ohlcv['Cam_S1_day_5'] 
stocks_daily_ohlcv['Cam_R3_day_1P_DIV_Cam_R3_day_1'] = stocks_daily_ohlcv['Cam_R3_day_1P'] / stocks_daily_ohlcv['Cam_R3_day_1']  
stocks_daily_ohlcv['Cam_S3_day_1P_DIV_Cam_S3_day_1'] = stocks_daily_ohlcv['Cam_S3_day_1P'] / stocks_daily_ohlcv['Cam_S3_day_1']  
                         
stocks_daily_ohlcv['Adj_Close_DIV_EMA_9'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['EMA_9'] 
stocks_daily_ohlcv['EMA_9_DIV_MA50'] = stocks_daily_ohlcv['EMA_9'] / stocks_daily_ohlcv['Adj_Close_MA50'] 
stocks_daily_ohlcv['MA50_DIV_MA150'] = stocks_daily_ohlcv['Adj_Close_MA50'] / stocks_daily_ohlcv['Adj_Close_MA150'] 
stocks_daily_ohlcv['MA150_DIV_MA200'] = stocks_daily_ohlcv['Adj_Close_MA150'] / stocks_daily_ohlcv['Adj_Close_MA200']           

stocks_daily_ohlcv['Close_DIV_High_52_weeks'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['High_52_weeks']  
stocks_daily_ohlcv['Close_DIV_Low_52_weeks'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['Low_52_weeks']  

stocks_daily_ohlcv['Close_DIV_High_4_weeks'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['High_4_weeks']  
stocks_daily_ohlcv['Close_DIV_Low_4_weeks'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['Low_4_weeks']  

stocks_daily_ohlcv['Close_DIV_EMA_10'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['EMA_10']
stocks_daily_ohlcv['Close_DIV_EMA_20'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['EMA_20']
stocks_daily_ohlcv['Close_DIV_MA50'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['Adj_Close_MA50']
stocks_daily_ohlcv['Close_DIV_MA200'] = stocks_daily_ohlcv['Adj_Close'] / stocks_daily_ohlcv['Adj_Close_MA200']

stocks_daily_ohlcv['EMA_5_DIV_EMA_20'] = stocks_daily_ohlcv['EMA_5'] / stocks_daily_ohlcv['EMA_20']

stocks_daily_ohlcv['Vol_Price'] = stocks_daily_ohlcv['Adj_Vol'] * stocks_daily_ohlcv['Adj_Close']


In [37]:
print(stocks_daily_ohlcv.info()) 
stocks_daily_ohlcv = hfs.convert_floats_to_float32(stocks_daily_ohlcv) 
stocks_daily_ohlcv.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755570 entries, 0 to 755569
Columns: 226 entries, company_Name to Vol_Price
dtypes: bool(9), datetime64[ns](1), float32(188), float64(23), int64(2), object(3)
memory usage: 715.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755570 entries, 0 to 755569
Columns: 226 entries, company_Name to Vol_Price
dtypes: bool(9), datetime64[ns](1), float32(211), int64(2), object(3)
memory usage: 649.2+ MB


In [38]:
print("#Tickers: ", str(stocks_daily_ohlcv['Ticker'].nunique()))
stocks_daily_ohlcv = stocks_daily_ohlcv.groupby(['Ticker']).apply(lambda x: x.iloc[290:]).reset_index(drop=True) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 

#Tickers:  1340
(366970, 226)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [39]:
missing_df = hfs.missing_values_info(stocks_daily_ohlcv) 
print(missing_df.shape) 
missing_df['Feature'].values 

(7, 3)


array(['Adj_Close_before_300d', 'body_per_3month_mean',
       'body_per_20d_mean', 'body_per_5d_mean', 'body_percent',
       'EMA_prev_5d_DIV_Adj_Vol', 'EMA_prev_20d_DIV_Adj_Vol'],
      dtype=object)

In [40]:
stocks_daily_ohlcv.fillna(0, inplace=True)

In [41]:
hfs.missing_values_info(stocks_daily_ohlcv)

,Feature,#missing_values,%missing_values


In [42]:
now = datetime.now() 
stocks_daily_ohlcv['Day_of_Week'] = stocks_daily_ohlcv['Date'].dt.weekday
stocks_daily_ohlcv['Day_of_Month'] = stocks_daily_ohlcv['Date'].dt.day
stocks_daily_ohlcv['Month_of_Year'] = stocks_daily_ohlcv['Date'].dt.month
print((datetime.now() - now).total_seconds()) 

0.02361


# indices data

In [43]:
indices_daily_ohlcv = pd.read_pickle(f"{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}/{[x for x in files_in_PROCESSING_TEMP if 'indices_daily_ohlcv' in x][0]}") 
indices_daily_ohlcv["Date"] = hfs.str_to_date_col(indices_daily_ohlcv[["Date"]]) 
print(indices_daily_ohlcv.shape) 
indices_daily_ohlcv = hfs.convert_floats_to_float32(indices_daily_ohlcv)
indices_daily_ohlcv.head(2) 

(1027, 61)


,Date,Adj_Open_GSPC,Adj_High_GSPC,Adj_Low_GSPC,Adj_Close_GSPC,Adj_Vol_GSPC,change_last_1d_GSPC,change_last_2d_GSPC,change_last_5d_GSPC,change_last_10d_GSPC,points_change_GSPC,change_last_1d_mean10_GSPC,Adj_Vol_1d_GSPC,Adj_Vol_EMA_prev_5d_GSPC,Adj_Vol_EMA_prev_20d_GSPC,EMA_prev_20d_DIV_Adj_Vol_GSPC,EMA_prev_5d_DIV_Adj_Vol_GSPC,Adj_Close_MA50_GSPC,Adj_Close_MA150_GSPC,Adj_Close_MA200_GSPC,BB_2_up_10_Adj_Close_GSPC,BB_2_dn_10_Adj_Close_GSPC,BB_2_up_20_Adj_Close_GSPC,BB_2_dn_20_Adj_Close_GSPC,RSI_9_GSPC,RSI_14_GSPC,ADR_percent_5d_GSPC,ADR_percent_20d_GSPC,High_52_weeks_GSPC,Low_52_weeks_GSPC,High_26_weeks_GSPC,Low_26_weeks_GSPC,High_4_weeks_GSPC,Low_4_weeks_GSPC,High_1_week_GSPC,Low_1_week_GSPC,EMA_5_GSPC,EMA_10_GSPC,EMA_20_GSPC,Adj_Close_DIV_EMA_10_GSPC,EMA_10_DIV_MA50_GSPC,Close_DIV_High_52_weeks_GSPC,Close_DIV_Low_52_weeks_GSPC,Close_DIV_High_4_weeks_GSPC,Close_DIV_Low_4_weeks_GSPC,Close_DIV_EMA_10_GSPC,Close_DIV_EMA_20_GSPC,Close_DIV_MA50_GSPC,Close_DIV_MA200_GSPC,EMA_5_DIV_EMA_20_GSPC,Vol_Price_GSPC,ADR_5D_GSPC,ADR_5D_70per_GSPC,ADR_5D_95per_GSPC,ADR_20D_GSPC,ADR_20D_70per_GSPC,ADR_20D_95per_GSPC,close_high_avg_GSPC,stop_EMA20_GSPC,max_entry_stop_EMA20_ADR_20D_95per_GSPC,close_high_avg_inline_GSPC
0,2019-07-12,3003.360107,3013.919922,3001.870117,3013.770020,2.974960e+09,0.462,0.692,0.781,3.038,13.86,0.300,3.154620e+09,2.935166e+09,3.326478e+09,1.118,0.987,2891.379883,2958.509033,3032.606934,3021.562988,2943.968994,3025.667969,2879.791016,75.394997,72.717003,0.490,0.614,3393.52002,2191.860107,3085.199951,2728.810059,3013.919922,2879.620117,3013.919922,2963.439941,2996.601074,2981.352051,2954.525879,1.011,1.031,0.846,1.309,1.000,1.047,1.011,1.020,1.042,1.115,1.014,8.965845e+12,14.767,10.337,14.029,18.504999,12.953,17.579,3013.844971,2939.752930,2957.332031,False
1,2019-07-15,3017.800049,3017.800049,3008.770020,3014.300049,2.874970e+09,0.018,0.480,1.289,2.466,0.53,0.245,2.974960e+09,3.043316e+09,3.321735e+09,1.155,1.059,2893.315918,2958.509033,3032.606934,3021.956055,2958.083984,3030.168945,2888.021973,75.509003,72.801003,0.478,0.603,3393.52002,2191.860107,3085.199951,2728.810059,3017.800049,2887.300049,3017.800049,2963.439941,3002.500977,2987.343018,2960.253906,1.009,1.032,0.846,1.309,0.999,1.044,1.009,1.018,1.042,1.115,1.014,8.666022e+12,14.408,10.086,13.688,18.176001,12.723,17.267,3016.050049,2945.451904,2962.719971,False


In [44]:
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv = pd.merge(stocks_daily_ohlcv, indices_daily_ohlcv, how='left', on=['Date'])
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.tail(2) 

(366970, 229)
(366970, 289)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [45]:
missing_df = hfs.missing_values_info(stocks_daily_ohlcv) 
print(missing_df.shape) 
missing_df 

(0, 3)


,Feature,#missing_values,%missing_values


## industry Data

In [46]:
stocks_current_fundamentals = pd.read_pickle(f"{PROJ_PATH}/{ML_DATA_FOLDER_PATH}/{[x for x in files_in_ML_DATA if 'stocks_current_fundamentals' in x][0]}") 
print(stocks_current_fundamentals.shape) 
stocks_current_fundamentals.head(3) 


(1340, 10)


,Ticker,General__AddressData.Country,General__Sector,Highlights__EPSEstimateCurrentQuarter,Highlights__ReturnOnEquityTTM,Valuation__PriceBookMRQ,SharesStats__PercentInsiders,SharesStats__PercentInstitutions,SharesStats__SharesFloat,SharesStats__PercentRetailers
0,WMB,United States,Energy,0.38,0.2037,3.6408,0.356,87.527,1211.263836,12.117
1,FUTU,Hong Kong,Financial Services,0.82,0.1667,3.9834,16.407,39.594,56.984457,43.999
2,DRUG,Other,Healthcare,0.00,-1.0154,2.7138,23.666,3.006,2.993403,73.328


In [47]:
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv = pd.merge(stocks_daily_ohlcv, stocks_current_fundamentals[['Ticker','General__AddressData.Country','General__Sector',
                                                                                     'SharesStats__SharesFloat']], 
                                    how='left', on = "Ticker") 

print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 

(366970, 289)
(366970, 292)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [49]:
industry_df = stocks_daily_ohlcv.groupby(['Date','General__Sector']).agg({
    'Adj_Close_change_per_3d' : 'mean',
    'Adj_Close_change_per_20d' : 'mean',
    'ADR_percent_5d' : 'mean',
    'ADR_percent_20d' : 'mean',
    'Adj_Vol_change_per_3d' : 'mean',
    'Adj_Vol_change_per_20d' : 'mean',
    'Close_DIV_High_52_weeks' : 'mean',
    'Close_DIV_Low_52_weeks' : 'mean',
    'Close_DIV_EMA_20' : 'mean',
    'Close_DIV_MA200' : 'mean',
    'Close_DIV_MA50' : 'mean',
    'Ticker' : 'count'
}).reset_index()
industry_df.columns = ['Date', 'General__Sector'] + [f"ind_{x}" for x in industry_df.columns[2:]]

for col in industry_df.columns[2:-1]:
    industry_df[col].fillna(np.median(sorted([x for x in industry_df[col].values if ~np.isnan(x)])), inplace=True)

print(industry_df.shape)
industry_df.head() 

(3377, 14)


,Date,General__Sector,ind_Adj_Close_change_per_3d,ind_Adj_Close_change_per_20d,ind_ADR_percent_5d,ind_ADR_percent_20d,ind_Adj_Vol_change_per_3d,ind_Adj_Vol_change_per_20d,ind_Close_DIV_High_52_weeks,ind_Close_DIV_Low_52_weeks,ind_Close_DIV_EMA_20,ind_Close_DIV_MA200,ind_Close_DIV_MA50,ind_Ticker
0,2022-06-27,,-0.019000,-0.135000,7.447000,7.693000,0.425000,-0.272000,0.164000,1.062000,0.938000,0.574000,0.846000,1
1,2022-06-27,Basic Materials,0.028182,-0.117200,5.915109,5.865400,-0.037073,0.249509,0.538691,1.344582,0.961327,0.805909,0.887709,55
2,2022-06-27,Communication Services,0.082469,-0.024922,8.074203,7.236828,0.467500,11.367359,0.378328,1.356375,1.002391,0.655797,0.940391,64
3,2022-06-27,Consumer Cyclical,0.046085,-0.040907,6.630612,6.554605,0.216279,0.148574,0.396969,1.288395,0.988682,0.659217,0.918271,129
4,2022-06-27,Consumer Defensive,0.058342,0.081868,7.649421,7.125237,1.777579,7.544000,0.490342,1.517711,1.028053,0.827921,1.009947,38


In [50]:
missing_df = hfs.missing_values_info(industry_df) 
print(missing_df.shape) 
missing_df 

(0, 3)


,Feature,#missing_values,%missing_values


In [51]:
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv = pd.merge(stocks_daily_ohlcv, industry_df, 
                                    how='left', on = ['Date', 'General__Sector']) 

print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.tail(2) 

(366970, 292)
(366970, 304)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [54]:
stocks_daily_ohlcv['ticker_DIV_SPY_ADR_5D_per'] = stocks_daily_ohlcv['ADR_percent_5d'] / stocks_daily_ohlcv['ADR_percent_5d_GSPC'] 
stocks_daily_ohlcv['ticker_DIV_SPY_ADR_20D_per'] = stocks_daily_ohlcv['ADR_percent_20d'] / stocks_daily_ohlcv['ADR_percent_20d_GSPC'] 

stocks_daily_ohlcv['ticker_DIV_industry_ADR_5D_per'] = stocks_daily_ohlcv['ADR_percent_5d'] / stocks_daily_ohlcv['ind_ADR_percent_5d'] 
stocks_daily_ohlcv['ticker_DIV_industry_ADR_20D_per'] = stocks_daily_ohlcv['ADR_percent_20d'] / stocks_daily_ohlcv['ind_ADR_percent_20d'] 


In [55]:
col_type = stocks_daily_ohlcv.dtypes.reset_index()
col_type.columns = ['Column_Name', 'type']
float_columns = list(col_type["Column_Name"][(col_type['type']=="float64") |
                                             (col_type['type']=="float32") 
                                            ].values)
len(float_columns) 


286

In [56]:
cols_is_finite = np.isfinite(stocks_daily_ohlcv[float_columns]).all().reset_index()
cols_is_finite.columns = ['Column_name', 'is_finite'] 
print("cols with inf values : ", len(list(cols_is_finite["Column_name"][cols_is_finite['is_finite']==False].values)) )

# replacing infinite values with five times max/min values with considering sign (negative or positive)
for col in list(cols_is_finite["Column_name"][cols_is_finite['is_finite']==False].values):
    stocks_daily_ohlcv[col] = np.where(np.isfinite(stocks_daily_ohlcv[col]),
                                    stocks_daily_ohlcv[col],
                                    5 * np.sign(stocks_daily_ohlcv[col]) * \
                        np.max(np.abs(stocks_daily_ohlcv[col][np.isfinite(stocks_daily_ohlcv[col])]))
                                   ) 

cols with inf values :  18


In [57]:
stocks_daily_ohlcv.tail(2)

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [58]:
for col in indices_daily_ohlcv.columns[1:]:
    stocks_daily_ohlcv[col] = stocks_daily_ohlcv.groupby(['Ticker'])[col].transform(lambda x: x.ffill() )
    stocks_daily_ohlcv[col] = stocks_daily_ohlcv.groupby(['Ticker'])[col].transform(lambda x: x.bfill() )


In [59]:
dates_with_no_gspc = stocks_daily_ohlcv[pd.isnull(stocks_daily_ohlcv['Adj_Close_GSPC'])]['Date'].unique()
ticker_with_no_gspc = stocks_daily_ohlcv[pd.isnull(stocks_daily_ohlcv['Adj_Close_GSPC'])]['Ticker'].unique()
if len(dates_with_no_gspc)>0:
    print(adflajdflajf)
if len(ticker_with_no_gspc)>0:
    print(adflajdflajf)
    

In [60]:
missing_df = hfs.missing_values_info(stocks_daily_ohlcv) 
print(missing_df.shape) 
missing_df 


(0, 3)


,Feature,#missing_values,%missing_values


In [61]:
if len(missing_df)>0:
    print(adflajdflajf)
    

In [62]:
stocks_daily_ohlcv['Exchange'] = stocks_daily_ohlcv['Exchange'].astype('category') 

price_cat_conditions = [ (stocks_daily_ohlcv['Adj_Close'] <2) ,
                         (stocks_daily_ohlcv['Adj_Close'] >=2) & (stocks_daily_ohlcv['Adj_Close']<5),
                         (stocks_daily_ohlcv['Adj_Close'] >=5) & (stocks_daily_ohlcv['Adj_Close']<20),
                         (stocks_daily_ohlcv['Adj_Close'] >=20) 
                        ]
 
price_cat_choices = ['LT_2',
                  'GT_2_LT_5', 
                  'GT_5_LT_20', 
                  'GT_20']

stocks_daily_ohlcv['price_cat'] = np.select(price_cat_conditions, price_cat_choices, default= "price_" )
stocks_daily_ohlcv['price_cat'] = stocks_daily_ohlcv['price_cat'].astype('category') 
print(stocks_daily_ohlcv.shape)
stocks_daily_ohlcv.head(2) 

(366970, 309)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [63]:
stocks_daily_ohlcv = stocks_daily_ohlcv[-stocks_daily_ohlcv['Ticker'].isin(['BIPH','DNAD', 'GRPH', 'SPIR', 'ZWZZT', 'ZVZZT'
                                                                                            ])].reset_index(drop=True) 
stocks_daily_ohlcv.shape


(366970, 309)

In [64]:
stocks_daily_ohlcv.tail(2)

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [65]:
#stocks_daily_ohlcv.rename(columns={'Adj_Close_x' : 'Adj_Close'}, inplace=True)

## Analytics df

In [67]:
analytics_feature_ind = stocks_daily_ohlcv[stocks_daily_ohlcv['Date']>=(SPLIT_DATE - timedelta(days=370))].reset_index(drop=True)
print(analytics_feature_ind.shape)
analytics_feature_ind.to_pickle(f'{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}/analytics_feature_ind.pkl')


(334383, 309)


In [68]:
# converting 0/1 columns to boolean type
bool_cols_01_vals = []
for col in stocks_daily_ohlcv.columns:
    #print(col)
    unique_vals = stocks_daily_ohlcv[col].unique()
    if (len(unique_vals) == 2):
        if sorted(unique_vals) == [0,1]:
            bool_cols_01_vals.append(col)
            stocks_daily_ohlcv[col] = stocks_daily_ohlcv[col].astype(bool)

print("Bool cols: ", len(bool_cols_01_vals))
stocks_daily_ohlcv[bool_cols_01_vals].tail(2) 

Bool cols:  10


,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,straight_down,straight_up,close_high_avg_inline_GSPC
366968,False,False,False,False,False,False,False,False,False,True
366969,False,False,False,False,False,False,False,False,True,True


In [69]:
no_change_cols = []
for col in stocks_daily_ohlcv.columns:
    if stocks_daily_ohlcv[col].nunique() == 1:
        no_change_cols.append(col)

print("These columns are useless so removing: ", no_change_cols) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.drop(columns=no_change_cols, inplace=True) 
print(stocks_daily_ohlcv.shape) 

These columns are useless so removing:  []
(366970, 309)
(366970, 309)


In [70]:
stocks_daily_ohlcv.head(2)

,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,Exchange,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Clos

In [71]:
stocks_daily_ohlcv['General__Sector'] = stocks_daily_ohlcv['General__Sector'].astype('category') 
stocks_daily_ohlcv['General__AddressData.Country'] = stocks_daily_ohlcv['General__AddressData.Country'].astype('category') 

categorical_data = stocks_daily_ohlcv.select_dtypes(include=['category']) 
print(categorical_data.shape) 
categorical_data.head(2) 



(366970, 4)


,Exchange,General__AddressData.Country,General__Sector,price_cat
0,NYSE,United States,Basic Materials,GT_20
1,NYSE,United States,Basic Materials,GT_20


In [72]:
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv = pd.get_dummies(stocks_daily_ohlcv, columns=[x for x in categorical_data.columns], 
                                                   prefix=[x for x in categorical_data.columns]) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.head(2) 

(366970, 309)
(366970, 331)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Close_DIV_EMA

In [73]:
missing_df = hfs.missing_values_info(stocks_daily_ohlcv) 
print(missing_df.shape) 
missing_df 


(0, 3)


,Feature,#missing_values,%missing_values


In [74]:
print("#Tickers: ", str(stocks_daily_ohlcv['Ticker'].nunique())) 
print(stocks_daily_ohlcv.shape) 
stocks_daily_ohlcv.head(2) 


#Tickers:  1340
(366970, 331)


,company_Name,Ticker,Date,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Vol,OHLC_price,Adj_Close_before_1d,Adj_Close_before_2d,Adj_Close_before_3d,Adj_Close_before_5d,Adj_Close_before_10d,Adj_Close_before_20d,Adj_Close_before_120d,Adj_Close_before_240d,Adj_Close_before_300d,Adj_Close_change_per_1d,Adj_Close_change_per_2d,Adj_Close_change_per_3d,Adj_Close_change_per_5d,Adj_Close_change_per_10d,Adj_Close_change_per_20d,Adj_Close_change_per_120d,Adj_Close_change_per_240d,Adj_Close_change_per_300d,Adj_Close_change_per_1d_mean10,Cam_Range_1_days,Cam_PP_day_1,Cam_R3_day_1,Cam_S3_day_1,Cam_Range_1P_days,Cam_PP_day_1P,Cam_R3_day_1P,Cam_S3_day_1P,Cam_PP_day_1_per,Cam_R3_day_1_per,Cam_S3_day_1_per,Cam_PP_day_1P_per,Cam_R3_day_1P_per,Cam_S3_day_1P_per,Cam_Range_5_days,Cam_PP_day_5,Cam_R1_day_5,Cam_S1_day_5,Cam_Range_5P_days,Cam_PP_day_5P,Cam_R1_day_5P,Cam_S1_day_5P,Cam_PP_day_5_per,Cam_R1_day_5_per,Cam_S1_day_5_per,Cam_PP_day_5P_per,Cam_R1_day_5P_per,Cam_S1_day_5P_per,Adj_High_2_days_per,Adj_Low_2_days_per,Cam_PP_day_2_per,Cam_R1_day_2_per,Cam_S1_day_2_per,Adj_High_20_days_per,Adj_Low_20_days_per,Cam_PP_day_20_per,Cam_R1_day_20_per,Cam_S1_day_20_per,Adj_High_120_days_per,Adj_Low_120_days_per,Cam_PP_day_120_per,Cam_R1_day_120_per,Cam_S1_day_120_per,Adj_High_240_days_per,Adj_Low_240_days_per,Cam_PP_day_240_per,Cam_R1_day_240_per,Cam_S1_day_240_per,RSI_9,RSI_14,RSI_20,volatility_126,max_drop_126,volatility_20,max_drop_20,TR,ATR_3,volatility_5,volatility_3,volatility_2,max_drop_5,max_drop_3,ADR_percent_5d,ADR_percent_20d,BB_2_up_20_Adj_Close,BB_2_dn_20_Adj_Close,BB_3_up_20_Adj_Close,BB_3_dn_20_Adj_Close,Adj_Close_lag1,Adj_Close_MA200,Adj_Close_MA150,Adj_Close_MA100,Adj_Close_MA90,Adj_Close_MA50,Adj_Close_MA40,Adj_Close_MA26,Adj_Close_MA20,Adj_Close_MA15,Adj_Close_MA12,Adj_Close_MA10,Adj_Close_MA5,Adj_Close_MA3,Adj_Close_MA2,D1_Close_volatility_percnt,D1_Close_volatility_percnt_before_1d,D1_Close_volatility_percnt_before_2d,D1_Close_volatility_percnt_before_3d,D1_Close_volatility_percnt_before_5d,D1_Close_volatility_percnt_before_10d,D1_Close_volatility_percnt_before_20d,price_voilatility_change_last_1d,price_voilatility_change_last_2d,price_voilatility_change_last_3d,price_voilatility_change_last_5d,price_voilatility_change_last_10d,price_voilatility_change_last_20d,gap_percent,gap_5day_max,gap_5day_min,High_52_weeks,Low_52_weeks,High_26_weeks,Low_26_weeks,High_4_weeks,Low_4_weeks,High_1_week,Low_1_week,body_percent,body_per_5d_mean,body_per_20d_mean,body_per_3month_mean,Long_MA20_crossed_MA200_in_5d,Short_MA20_crossed_MA200_in_5d,Long_MA10_crossed_MA50_in_5d,Short_MA10_crossed_MA50_in_2d,Long_MA20_crossed_MA200_in_2d,Short_MA20_crossed_MA200_in_2d,Long_MA10_crossed_MA50_in_2d,EMA_5,EMA_9,EMA_10,EMA_20,ATR,KCupper,KClower,High_Open_diff,Low_Open_diff,High_Low_diff,High_Open_diff_5d_max,High_Open_diff_5d_min,Low_Open_diff_5d_max,Low_Open_diff_5d_min,High_Low_diff_5d_max,High_Low_diff_5d_min,straight_down,straight_up,straight_down_per,straight_up_per,Close_Low_diff,Close_Open_diff,straight_up_close_per,Adj_Vol_lag1,Adj_Vol_MA60,Adj_Vol_MA20,Adj_Vol_MA10,Adj_Vol_MA5,Adj_Vol_MA2,Adj_Vol_STD20,Adj_Vol_STD10,Adj_Vol_STD5,Adj_Vol_STD2,On_Balance_Adj_Vol,Adj_Vol_before_1d,Adj_Vol_before_2d,Adj_Vol_before_3d,Adj_Vol_before_5d,Adj_Vol_before_10d,Adj_Vol_before_20d,Adj_Vol_change_per_1d,Adj_Vol_change_per_2d,Adj_Vol_change_per_3d,Adj_Vol_change_per_5d,Adj_Vol_change_per_10d,Adj_Vol_change_per_20d,VWAP_Adj_Close5,VWAP_Adj_Close3,VWAP_Adj_Close2,5_day_negatives,10_day_negatives,Adj_Vol_1d,Adj_Vol_EMA_prev_5d,Adj_Vol_EMA_prev_20d,EMA_prev_20d_DIV_Adj_Vol,EMA_prev_5d_DIV_Adj_Vol,Opening_Gap_Percent,day_Open_high_per,day_Open_low_per,day_Open_close_per,ATR_3_DIV_OHLC_price,Cam_R1_day_5P_DIV_Cam_R1_day_5,Cam_S1_day_5P_DIV_Cam_S1_day_5,Cam_R3_day_1P_DIV_Cam_R3_day_1,Cam_S3_day_1P_DIV_Cam_S3_day_1,Adj_Close_DIV_EMA_9,EMA_9_DIV_MA50,MA50_DIV_MA150,MA150_DIV_MA200,Close_DIV_High_52_weeks,Close_DIV_Low_52_weeks,Close_DIV_High_4_weeks,Close_DIV_Low_4_weeks,Close_DIV_EMA

In [75]:
stocks_daily_ohlcv.to_pickle(f'{PROJ_PATH}/{PROCESSING_TEMP_FOLDER_PATH}/OHLCV_all_stocks_ohlcv_data_prep.pkl') 


In [76]:
%reset -f 